# Generative Adversarial Networks

Transposed Convolutions

In [1]:
import torch as t
import torch.nn as nn
import utils
import past
import einops
import typing
from typing import Union

In [2]:
def conv_transpose1d_minimal(x: t.Tensor, weights: t.Tensor) -> t.Tensor:
    '''Like torch's conv_transpose1d using bias=False and all other keyword arguments left at their default values.

    x: shape (batch, in_channels, width)
    weights: shape (in_channels, out_channels, kernel_width)

    Returns: shape (batch, out_channels, output_width)
    '''

    pad_size = weights.shape[2] - 1
    pad_x = past.pad1d(x, pad_size, pad_size, 0)
    kernel_mod = weights.flip(-1)
    kernel_mod = einops.rearrange(kernel_mod, 'in_channels out_channels kernel_width -> out_channels in_channels kernel_width')
    return past.conv1d_minimal(pad_x, kernel_mod)


utils.test_conv_transpose1d_minimal(conv_transpose1d_minimal)

All tests in `test_conv1d_minimal` passed!


In [3]:
def fractional_stride_1d(x, stride: int = 1):
    '''Returns a version of x suitable for transposed convolutions, i.e. "spaced out" with zeros between its values.
    This spacing only happens along the last dimension.

    x: shape (batch, in_channels, width)

    Example: 
        x = [[[1, 2, 3], [4, 5, 6]]]
        stride = 2
        output = [[[1, 0, 2, 0, 3], [4, 0, 5, 0, 6]]]
    '''

    batch, in_channels, width = x.shape
    new_width = (width - 1) * stride + 1
    new_x = t.zeros((batch, in_channels, new_width), dtype=x.dtype)

    new_x[:, :, ::stride] = x

    return new_x

utils.test_fractional_stride_1d(fractional_stride_1d)

All tests in `test_fractional_stride_1d` passed!


In [4]:
def conv_transpose1d(x, weights, stride: int = 1, padding: int = 0) -> t.Tensor:
    '''Like torch's conv_transpose1d using bias=False and all other keyword arguments left at their default values.

    x: shape (batch, in_channels, width)
    weights: shape (in_channels, out_channels, kernel_width)

    Returns: shape (batch, out_channels, output_width)
    '''

    #print(x[0])
    new_x = fractional_stride_1d(x, stride)
    #print(new_x[0])

    pad_size = weights.shape[2] - 1 - padding
    pad_x = past.pad1d(new_x, pad_size, pad_size, 0)
    kernel_mod = weights.flip(-1)
    kernel_mod = einops.rearrange(kernel_mod, 'in_channels out_channels kernel_width -> out_channels in_channels kernel_width')
    
    pad_x = pad_x.float()
    return past.conv1d_minimal(pad_x, kernel_mod)
    
utils.test_conv_transpose1d(conv_transpose1d)

All tests in `test_conv_transpose1d` passed!


In [5]:
# testing it personally
to_trans = t.tensor([[[1.,5.,6.,7.,9.]]], dtype=t.float64)
conv_transpose1d(to_trans, t.tensor([[[1.,1.,1.]]], dtype=t.float32),padding=1)

tensor([[[ 6., 12., 18., 22., 16.]]])

In [6]:
IntOrPair = Union[int, tuple[int, int]]
Pair = tuple[int, int]

def force_pair(v: IntOrPair) -> Pair:
    '''Convert v to a pair of int, if it isn't already.'''
    if isinstance(v, tuple):
        if len(v) != 2:
            raise ValueError(v)
        return (int(v[0]), int(v[1]))
    elif isinstance(v, int):
        return (v, v)
    raise ValueError(v)

def fractional_stride_2d(x, stride_h: int, stride_w: int):
    '''
    Same as fractional_stride_1d, except we apply it along the last 2 dims of x (width and height).
    '''
    batch, in_channels, height, width = x.shape
    new_width = (width - 1) * stride_w + 1
    new_height = (height - 1) * stride_h + 1
    new_x = t.zeros((batch, in_channels, new_height, new_width))

    new_x[:, :, ::stride_h, ::stride_w] = x

    return new_x

def conv_transpose2d(x, weights, stride: IntOrPair = 1, padding: IntOrPair = 0) -> t.Tensor:
    '''Like torch's conv_transpose2d using bias=False

    x: shape (batch, in_channels, height, width)
    weights: shape (in_channels, out_channels, kernel_height, kernel_width)


    Returns: shape (batch, out_channels, output_height, output_width)
    '''
    stride: Pair = force_pair(stride)
    padding: Pair = force_pair(padding)

    #print(x[0])
    new_x = fractional_stride_2d(x, stride[0], stride[1])
    #print(new_x[0])

    pad_size_h = weights.shape[-2] - 1 - padding[0]
    pad_size_w = weights.shape[-1] - 1 - padding[1]
    pad_x = past.pad2d(new_x, pad_size_w, pad_size_w, pad_size_h, pad_size_h,  0)
    kernel_mod = weights.flip(-1)
    kernel_mod = kernel_mod.flip(-2)
    kernel_mod = einops.rearrange(kernel_mod, 'in_channels out_channels kernel_height kernel_width -> out_channels in_channels kernel_height kernel_width')

    pad_x = pad_x.float()
    return past.conv2d_minimal(pad_x, kernel_mod)
    pass

utils.test_conv_transpose2d(conv_transpose2d)

All tests in `test_conv_transpose2d` passed!


## Other Modules

In [7]:
class ConvTranspose2d(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int, kernel_size: IntOrPair, stride: IntOrPair = 1, padding: IntOrPair = 0
    ):
        '''
        Same as torch.nn.ConvTranspose2d with bias=False.

        Name your weight field `self.weight` for compatibility with the tests.
        '''


            # x: shape (batch, in_channels, height, width)
            # weights: shape (out_channels, in_channels, kernel_height, kernel_width)
            # returns: shape (batch, out_channels, output_height, output_width)


        super().__init__()
        kernel_size = force_pair(kernel_size)
        k = (1 / (out_channels * kernel_size[0] * kernel_size[1])) ** 0.5
        self.weight = nn.Parameter(t.rand((in_channels, out_channels, kernel_size[0], kernel_size[1])) * (k * 2) - k)
        # print(self.weight.shape)
        self.stride = force_pair(stride)
        self.padding = force_pair(padding)

    def forward(self, x: t.Tensor) -> t.Tensor:
        return conv_transpose2d(x, self.weight, self.stride, self.padding)

utils.test_ConvTranspose2d(ConvTranspose2d)

torch.Size([5, 8, 43, 47])
torch.Size([8, 1, 1, 2])
------
torch.Size([5, 1, 85, 46])
torch.Size([5, 1, 85, 46])
torch.Size([6, 7, 30, 28])
torch.Size([7, 4, 5, 3])
------
torch.Size([6, 4, 117, 80])
torch.Size([6, 4, 117, 80])
torch.Size([4, 7, 41, 15])
torch.Size([7, 5, 3, 6])
------
torch.Size([4, 5, 81, 44])
torch.Size([4, 5, 81, 44])
torch.Size([5, 9, 40, 45])
torch.Size([9, 2, 5, 7])
------
torch.Size([5, 2, 75, 175])
torch.Size([5, 2, 75, 175])
torch.Size([1, 11, 32, 27])
torch.Size([11, 2, 3, 2])
------
torch.Size([1, 2, 63, 26])
torch.Size([1, 2, 63, 26])
All tests in `test_ConvTranspose2d` passed!


In [8]:
class Tanh(nn.Module):
    def forward(self, x: t.Tensor) -> t.Tensor:
        return (t.e ** x - t.e ** (-1 * x)) / (t.e ** x + t.e ** (-1 * x))

utils.test_Tanh(Tanh)

All tests in `test_Tanh` passed.


In [9]:
class LeakyReLU(nn.Module):
    def __init__(self, negative_slope: float = 0.01):
        super().__init__()
        self.negslope = negative_slope

    def forward(self, x: t.Tensor) -> t.Tensor:
        return t.maximum(x, t.zeros(x.shape)) + self.negslope * t.minimum(x, t.zeros(x.shape))

    def extra_repr(self) -> str:
        pass

utils.test_LeakyReLU(LeakyReLU)

All tests in `test_LeakyReLU` passed.


In [10]:
class Sigmoid(nn.Module):
    def forward(self, x: t.Tensor) -> t.Tensor:
        return 1 / (1 + t.e ** (-1 * x))

utils.test_Sigmoid(Sigmoid)

All tests in `test_Sigmoid` passed.
